In [3]:
%cd model/

c:\Users\sduai\Desktop\Batyrbayev_Adilet\qCloudy\Projects\PoC\Saiga\Saiga-cv\src\test\deploy\model


In [6]:
!python -m tarfile -c model.tar.gz 

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

session = boto3.Session(profile_name='saiga', region_name='eu-north-1')
sagemaker_session = sagemaker.Session(boto_session=session)
role = get_execution_role(sagemaker_session)
sagemaker_client = session.client('sagemaker')

print(f"Role ARN: {role}")
print("SageMaker session initialized successfully!")


Couldn't call 'get_role' to get Role ARN from role name qcloudy-adilet to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::112557133449:user/qcloudy-adilet, therefore it cannot be used as a SageMaker execution role

In [1]:
!uv add boto3 sagemaker

Resolved 107 packages in 1ms
Audited 85 packages in 4ms


In [ ]:
!uv sync

Resolved 107 packages in 1ms
Audited 85 packages in 6ms


In [8]:
import boto3

session = boto3.Session(profile_name='saiga')

s3_client = session.client('s3')
bucket_name = 'saiga-prod-model-weights'
model_artifacts = 'model/model.tar.gz'
s3_key = 'yolo/model.tar.gz'

s3_client.upload_file(model_artifacts, bucket_name, s3_key)
model_data = f's3://{bucket_name}/{s3_key}'
print(f"Model uploaded to: {model_data}")

Model uploaded to: s3://saiga-prod-model-weights/yolo/model.tar.gz


In [11]:
aws ecr get-login-password --region eu-north-1 --profile saiga | docker login --username AWS --password-stdin 112557133449.dkr.ecr.eu-north-1.amazonaws.com

SyntaxError: invalid decimal literal (2815040013.py, line 1)

In [ ]:
aws ecr create-repository --repository-name saiga-yolo-model --profile saiga

In [ ]:
docker build -t saiga-yolo-model .

In [ ]:
docker tag saiga-yolo-model:latest 112557133449.dkr.ecr.eu-north-1.amazonaws.com/saiga-yolo-model:latest
docker push 112557133449.dkr.ecr.eu-north-1.amazonaws.com/saiga-yolo-model:latest

In [12]:
import boto3
import sagemaker
from sagemaker import get_execution_role
session = boto3.Session(profile_name='saiga', region_name='eu-north-1')

sts = session.client('sts')
assumed = sts.assume_role(
    RoleArn='arn:aws:iam::112557133449:role/saiga-prod-sagemaker-exec-role',
    RoleSessionName='sagemaker-session'
)
creds = assumed['Credentials']

sagemaker_client = boto3.client(
    'sagemaker',
    aws_access_key_id=creds['AccessKeyId'],
    aws_secret_access_key=creds['SecretAccessKey'],
    aws_session_token=creds['SessionToken'],
    region_name='eu-north-1'
)

container_image = '112557133449.dkr.ecr.eu-north-1.amazonaws.com/saiga-yolo-model:latest'

# Or use PyTorch container
# from sagemaker.pytorch import PyTorchModel
# pytorch_version = '2.0'
# container_image = sagemaker.image_uris.retrieve('pytorch', region, pytorch_version)

model_name = 'yolo-custom-model'

create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    PrimaryContainer={
        'Image': container_image,
        'ModelDataUrl': model_data,
        'Environment': {
            'SAGEMAKER_PROGRAM': 'inference.py',
            'SAGEMAKER_SUBMIT_DIRECTORY': model_data
        }
    },
    ExecutionRoleArn="arn:aws:iam::112557133449:role/saiga-prod-sagemaker-exec-role"
)

print(f"Model created: {create_model_response['ModelArn']}")

ClientError: An error occurred (ValidationException) when calling the CreateModel operation: Unsupported manifest media type application/vnd.oci.image.index.v1+json for image 112557133449.dkr.ecr.eu-north-1.amazonaws.com/saiga-yolo-model:latest. Ensure that valid manifest media type is used for specified image.